## Implementing Reflexion Agent

### Actor Agent

#### Schema

In [1]:
# Imports
from pydantic import BaseModel, Field
from typing import List

In [2]:
class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous.")

In [3]:
class AnswerQuestion(BaseModel):
    """Answer the questions"""
    answer: str = Field(description="250 word detailed answer to the question.")
    reflection: Reflection = Field(description="Your reflection on the initial answer.")
    queries: List[str] = Field(description="1-3 search queries for researching improvements to address the critique of your current answer.")

#### Chains

In [4]:
#imports
from datetime import datetime
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputToolsParser, PydanticToolsParser
from langchain_core.messages import HumanMessage


In [5]:
load_dotenv()
llm = ChatGroq(model="llama-3.3-70b-versatile")
llm=llm.bind_tools(tools=[AnswerQuestion],tool_choice="AnswerQuestion")

In [6]:
json_parser = JsonOutputToolsParser(return_id=True)
pydantic_parser = PydanticToolsParser(tools=[AnswerQuestion])

In [7]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",""" 
                    You are expert researcher.
                    Current time: {time}

                    1. {first_instruction}
                    2. Reflect and critique your answer. Be severe to maximize improvement.
                    3. Recommend search queries to research information and improve your answer.
                 """),
        MessagesPlaceholder(variable_name="messages"),
        ("system","Answer the user's question above using the required format.")
    ]
).partial(time=lambda: datetime.now().isoformat())

In [8]:
first_responder_prompt_template = actor_prompt_template.partial(first_instruction="Provide a detailed 200 word answer.")

In [9]:
human_message = HumanMessage(
    content="Write about AI-Powered SOC / autonomous soc  problem domain, " \
    "list startups that do that and raised capital.")

In [29]:
first_responder = (first_responder_prompt_template|llm|pydantic_parser)
# first_responder = (first_responder_prompt_template|llm)

In [30]:
response = first_responder.invoke(input={"messages":[human_message]})

In [31]:
from pprint import pprint
pprint(response)
# pprint(response[0].answer)
# pprint(response[0].reflection)
# pprint(response[0].queries)

[AnswerQuestion(answer='The AI-Powered Security Operations Center (SOC) or autonomous SOC is a problem domain that involves the use of Artificial Intelligence (AI) and Machine Learning (ML) to automate and enhance the capabilities of traditional SOCs. The goal of an AI-Powered SOC is to improve the efficiency and effectiveness of security operations by leveraging AI and ML to detect, respond, and prevent cyber threats in real-time. This is achieved through the use of advanced analytics, threat intelligence, and automation. Some startups that are working in this space and have raised capital include: Cyware Labs ($30 million), Securonix ($29 million), and Deep Instinct ($100 million). These startups are developing innovative solutions that utilize AI and ML to detect and respond to threats, predict and prevent breaches, and improve the overall security posture of organizations. The AI-Powered SOC market is expected to grow significantly in the coming years, driven by the increasing need

In [23]:
class ReviseAnswer(AnswerQuestion):
    "Revise your original answer to your question."
    references: List[str] = Field(description="Citations motivating your updated answer.")

In [74]:
revise_instructions = """
    Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        - You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.

"""

In [75]:
revisor = actor_prompt_template.partial(first_instruction=revise_instructions) | llm.bind_tools(tools=[ReviseAnswer],tool_choice="ReviseAnswer")

#### Tool Executor

In [12]:
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode
from langchain_core.tools import StructuredTool
from langchain_core.messages import AIMessage

In [13]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search,max_results=2)

In [15]:
def run_queries(search_queries: list[str], **kwargs):
    """Run the generated queries."""
    return tavily_tool.batch([{"query": query} for query in search_queries])


In [19]:
tool_node = ToolNode([run_queries])

In [ ]:
raw_response = first_responder

In [32]:
chaining_tool = first_responder|tool_node

In [35]:
# search_result = run_queries(response[0].queries)
search_result = chaining_tool.invoke(input={"messages":[human_message]})

ValueError: Last message is not an AIMessage

In [69]:
tool_response=tool_node.invoke(to_pass_in_tool['generations'][0][0])

ValueError: No message found in input